# 한국 부동산 시장 분석 및 투자 전략 수립

## 과제 1: 서울 인기구 vs 비인기구 부동산 시장 비교 분석
가격 변동 패턴 비교분석, 지역별 시장 특성 및 투자 매력도 평가

- 인기구: 거래액/거래량 기준 상위 3개구, 비인기구: 거래액/거래량 기준 하위 3개구

- 지역별 시장 특성 시각화
  - 거래 밀도, 평균 거래가, 변동성 지표 등등

- 투자 매력도 -> 변동성, 수익률 
  - 변동성: 표준편차로 확인, 수익률은 연초에 샀을 때 얼마나 올랐는지

In [51]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from IPython.display import display

warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

df_sales = pd.read_csv('./data/sales_clean.csv')
df_sales['계약일'] = pd.to_datetime(df_sales['계약일'], format='%Y%m%d')

In [55]:
# 인기구/비인기구 정의 - 최근 12개월 기준 누적 거래액 기준 상위 3개구 / 하위 3개구
df_sales_copy = df_sales.copy()
df_sales_copy = df_sales_copy[df_sales_copy['공공임대'].isna()]

today = pd.to_datetime('2025-07-01')
date_ref = today - pd.DateOffset(years=1)
filtered_year_df = df_sales_copy[(df_sales_copy['계약일'] > date_ref) & (df_sales_copy['계약일'] <= today)]

district_sales_stats = filtered_year_df.groupby('구').agg({
    '거래금액': ['count', 'sum']
}).round(0)

district_sales_stats.columns = ['거래건수', '총거래액']

popular = district_sales_stats.sort_values(by = ['총거래액', '거래건수'], ascending=[False, False]).head(3).index.tolist()
unpopular = district_sales_stats.sort_values(by = ['총거래액', '거래건수'], ascending=[True, False]).head(3).index.tolist()

print(f'인기구: {popular}\n비인기구: {unpopular}')

인기구: ['강남구', '송파구', '서초구']
비인기구: ['금천구', '강북구', '종로구']


In [ ]:
# 핵심 지표 추출
sales_popular_mask = df_sales_copy['구'].isin(popular + unpopular)
df_popular = df_sales_copy[sales_popular_mask]

## 1. 전체 평균 거래가, 평단가
avg_price_stats = df_popular.groupby('구').agg({
    '거래금액': 'mean',
    '평단가': 'mean'
}).round(0)
avg_price_stats.columns = ['평균거래가', '평균평단가']
# display(avg_price_stats)

## 2. 변동성
### 월별 데이터 생성
monthly_stats = df_popular.groupby(['구', '계약월']).agg({'거래금액': ['mean', 'count']}).reset_index()
monthly_stats.columns = ['구', '계약월', '월평균거래금액', '월별거래건수']
monthly_stats['계약월'] = pd.to_datetime(monthly_stats['계약월'], format='%Y%m')
monthly_stats = monthly_stats.sort_values(['구', '계약월'])

### 월별 수익률 확인
monthly_return = monthly_stats[['구','계약월','월평균거래금액']]
monthly_return['월별수익률'] = monthly_return.groupby('구')['월평균거래금액'].transform(lambda x: np.log(x / x.shift(1)))
# display(monthly_return.head())

### 변동성(월수익률의 표준편차)
monthly_return['연도'] = monthly_return['계약월'].dt.year
volatility = monthly_return.groupby(['구', '연도'])['월별수익률'].std().reset_index()
volatility = volatility.rename(columns={'월별수익률': '연도별변동성'})
# display(volatility.head())

## 3. 거래밀도 (월별 거래건수 / 인구수)

### 구별/월별 인구수 데이터
population_df = pd.read_csv('./data/등록인구(월별).csv', header=[0, 1])
population_df = population_df.drop(columns=[population_df.columns[0]])
population_df.columns = ['_'.join(filter(None, map(str, col))).strip() for col in population_df.columns.values]

population_df_long = population_df.melt(id_vars=['동별(2)_동별(2)'], var_name='계약월_항목', value_name='값')
population_df_long[['계약월','항목']] = population_df_long['계약월_항목'].str.split('_', expand=True)
population_df_long['계약월'] = pd.to_datetime(population_df_long['계약월'].str.replace(" ", "").str.replace(".", "-"), format='%Y-%m')

### Pivot
df_population = population_df_long.pivot_table(
    index=['동별(2)_동별(2)', '계약월'],
    columns='항목',
    values='값'
).reset_index()
df_population = df_population.rename(columns={
    '동별(2)_동별(2)': '구',
    '세대 (세대)': '세대수',
    '합계 (명)': '총인구'
})

population_popular_mask = df_population['구'].isin(popular + unpopular)
population_density = pd.merge(df_population[population_popular_mask], monthly_stats[['구','계약월','월별거래건수']], on=['구', '계약월'], how='left')

### 거래밀도 계산(세대수, 총인구별)
population_density['거래밀도_총인구'] = population_density['월별거래건수'] / population_density['총인구']
population_density['거래밀도_세대수'] = population_density['월별거래건수'] / population_density['세대수']
# display(population_density)

## 12개월 이동평균, 이동표준편차
monthly_stats = monthly_stats.sort_values(['구', '계약월'])

# 이동평균과 이동표준편차 계산 (월평균거래금액 기준)
monthly_stats['이동평균'] = monthly_stats.groupby('구')['월평균거래금액'].transform(lambda x: x.rolling(12).mean())
monthly_stats['이동표준편차'] = monthly_stats.groupby('구')['월평균거래금액'].transform(lambda x: x.rolling(12).std())
# display(monthly_stats.head(20))

,구,계약월,월평균거래금액,월별거래건수,이동평균,이동표준편차
0,강남구,2020-01-01,1.542549e+09,133,NaN,NaN
1,강남구,2020-02-01,1.611718e+09,238,NaN,NaN
2,강남구,2020-03-01,1.687000e+09,138,NaN,NaN
3,강남구,2020-04-01,1.948344e+09,147,NaN,NaN
4,강남구,2020-05-01,1.819770e+09,305,NaN,NaN
5,강남구,2020-06-01,1.869041e+09,746,NaN,NaN
6,강남구,2020-07-01,1.681634e+09,370,NaN,NaN
7,강남구,2020-08-01,1.964748e+09,229,NaN,NaN
8,강남구,2020-09-01,1.892284e+09,171,NaN,NaN
9,강남구,2020-10-01,1.863225e+09,193,NaN,NaN


In [62]:
df_sales_copy.head()

,구,동,전용면적,전용면적(평),계약일,계약월,건축년도,단지명,거래금액,매수자,매도자,평단가,공공임대
0,성북구,돈암동,84.980,25.71,2020-12-31,202012,2013,돈암동해피트리,820000000,-,-,31894205,NaN
1,용산구,한남동,240.305,72.69,2020-12-31,202012,2011,한남더힐,7100000000,-,-,97675058,NaN
2,성동구,금호동4가,84.880,25.68,2020-12-31,202012,2018,힐스테이트서울숲리버,1800000000,-,-,70093458,NaN
3,동대문구,제기동,104.220,31.53,2020-12-31,202012,1978,공성,595000000,-,-,18870917,NaN
4,용산구,도원동,84.920,25.69,2020-12-31,202012,2001,삼성래미안,1350000000,-,-,52549630,NaN
